# Using Dask Gateway

This cluster has been configured to use [Dask-Gateway](https://gateway.dask.org/). This notebook demonstrates how to use it.

## Creating a Cluster

We've done most of the configuration for you. All that remains for you is to either create a new cluster or connect to an existing one. You'll do this with a `dask_gateway.Gateway` object.

If the default settings are appropriate, you can create a new cluster without specifying any additional information.

In [1]:
from dask_gateway import Gateway
gateway = Gateway()

In [2]:
gateway.list_clusters()

[]

In [3]:
cluster = gateway.new_cluster()
cluster

`gateway.new_cluster()` created a new Dask cluster.
You can pass this cluster to a `dask.distributed.Client` to create a client
so that any computations using Dask will be executed on the cluster.

In [4]:
from distributed import Client

client = Client(cluster)
client

Client Scheduler: gateway://scheduler-public-prod-dask-gateway:8786/e4734451d7ee49e8b2c4cce8e12d63de Dashboard: https://hub.binder.pangeo.io/services/dask-gateway/gateway/clusters/e4734451d7ee49e8b2c4cce8e12d63de/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
import dask.array as da

arr = da.random.random((1000, 1000), chunks=100).persist()

By default, the cluster doesn't have any workers. You'll need to either set your cluster to adaptive mode or scale manually.

In adaptive mode, the cluster will automatically resize itself based on the workload.

In [6]:
cluster.adapt()

In [7]:
arr.compute()

array([[0.41298364, 0.61513828, 0.05112384, ..., 0.88961295, 0.70132173,
        0.75764944],
       [0.02945153, 0.49759649, 0.47365479, ..., 0.80762362, 0.62463201,
        0.97132144],
       [0.38630875, 0.03178683, 0.75149671, ..., 0.73019977, 0.76219149,
        0.82755677],
       ...,
       [0.79606957, 0.05826694, 0.24142754, ..., 0.88395979, 0.94352869,
        0.37159944],
       [0.33892708, 0.05966802, 0.06047319, ..., 0.49311593, 0.02735945,
        0.44053292],
       [0.29584718, 0.55084755, 0.54991833, ..., 0.8669495 , 0.36673151,
        0.99985807]])

The `arr.compute` call may take more or less time, depending on the state of the Kubernetes cluster. If the cluster has been idle recently, we may be starting up additional machines in the background to do your work.

Subsequent computations, now that those machines are around, will be much faster.

In [8]:
client.close()
cluster.close()

## Specifying options

We've configured things like the the number of cores per worker to be appropriate for the most common use cases. For additional control over the cluster, use the `gateway.cluster_options`.

In [9]:
options = gateway.cluster_options()
options

Box(children=(Box(children=(HTML(value='<h2>Cluster Options</h2>'), Box(children=(HTML(value="<p style='font-w…

You can either manually adjust these options in widget, or set it programatically.

In [10]:
options.worker_cores = 4

The "Image" is a URL to a Docker image with the environment that will be loaded on the Dask scheduler and workers. This determines the versions of libraries that are available). By default, this matches the image you're currently working in.

In [11]:
cluster = gateway.new_cluster(cluster_options=options)
cluster.scale(1)
cluster

Notice that this cluster has 4 cores per worker, as we requested in the `options`.